# google drive

因为尝试了google官方文档的从Google drive下载文件的方法不成功，所以这里使用[pyDrive](https://github.com/gsuitedevs/PyDrive)尝试一下。

先简单补充下OAuth 2.0 协议的内容，参考了：[理解OAuth 2.0](https://www.ruanyifeng.com/blog/2014/05/oauth_2_0.html),[10 分钟理解什么是 OAuth 2.0 协议](https://deepzz.com/post/what-is-oauth2-protocol.html)。OAuth 2.0 协议目的是为第三方应用颁发一个有时效性的令牌 token。使得第三方应用能够通过该令牌获取相关的资源。常见的场景就是：第三方登录。

举例说明：有一个"云冲印"的网站，可以将用户储存在Google的照片，冲印出来。用户为了使用该服务，必须让"云冲印"读取自己储存在Google上的照片。

问题是只有得到用户的授权，Google才会同意"云冲印"读取这些照片。那么，"云冲印"怎样获得用户的授权呢？

传统方法是，用户将自己的Google用户名和密码，告诉"云冲印"，后者就可以读取用户的照片了。这样的做法有诸如“"云冲印"为了后续的服务，会保存用户的密码，这样很不安全”等缺点。OAuth就是为了解决这些问题而诞生的。

整个授权协议的流程都将围绕着这些角色：

（1） Third-party application：第三方应用程序，本文中又称"客户端"（client），即上一节例子中的"云冲印"。

（2）HTTP service：HTTP服务提供商，本文中简称"服务提供商"，即上一节例子中的Google。

（3）Resource Owner：资源所有者，本文中又称"用户"（user）。

（4）User Agent：用户代理，本文中就是指浏览器。

（5）Authorization server：认证服务器，即服务提供商专门用来处理认证的服务器。

（6）Resource server：资源服务器，即服务提供商存放用户生成的资源的服务器。它与认证服务器，可以是同一台服务器，也可以是不同的服务器。

OAuth的作用就是让"客户端"安全可控地获取"用户"的授权，与"服务商提供商"进行互动。

OAuth的思路是在"客户端"与"服务提供商"之间，设置一个授权层（authorization layer）。"客户端"不能直接登录"服务提供商"，只能登录授权层，以此将用户与客户端区分开来。"客户端"登录授权层所用的令牌（token），与用户的密码不同。用户可以在登录的时候，指定授权层令牌的权限范围和有效期。

"客户端"登录授权层以后，"服务提供商"根据令牌的权限范围和有效期，向"客户端"开放用户储存的资料。OAuth 2.0的运行流程如下图。

![](TIM截图20200728073151.png)

（A）用户打开客户端以后，客户端要求用户给予授权。

（B）用户同意给予客户端授权。

（C）客户端使用上一步获得的授权，向认证服务器申请令牌。

（D）认证服务器对客户端进行认证以后，确认无误，同意发放令牌。

（E）客户端使用令牌，向资源服务器申请获取资源。

（F）资源服务器确认令牌无误，同意向客户端开放资源。

步骤B，用户怎样才能给于客户端授权是比较关键的，授权方式有多种，google drive用的是授权码模式。授权码模式（authorization code）是功能最完整、流程最严密的授权模式。它的特点就是通过客户端的后台服务器，与"服务提供商"的认证服务器进行互动。

![](TIM截图20200728073542.png)

它的步骤如下：

（A）用户访问客户端，后者将前者导向认证服务器。

（B）用户选择是否给予客户端授权。

（C）假设用户给予授权，认证服务器将用户导向客户端事先指定的"重定向URI"（redirection URI），同时附上一个授权码。

（D）客户端收到授权码，附上早先的"重定向URI"，向认证服务器申请令牌。这一步是在客户端的后台的服务器上完成的，对用户不可见。

（E）认证服务器核对了授权码和重定向URI，确认无误后，向客户端发送访问令牌（access token）和更新令牌（refresh token）。

在访问google drive过程中，首先要使用OAuth 2.0来获取Google APIs，可以参考：[Using OAuth 2.0 to Access Google APIs](https://developers.google.com/identity/protocols/oauth2)，可以试试：[OAuth 2.0 Playground](https://developers.google.com/oauthplayground/)。这里我们直接使用pydrive来帮助访问，所以直接参考：[PyDrive Quickstart](https://pythonhosted.org/PyDrive/quickstart.html)

浏览器进入google cloud console，如果没有google cloud console，那就需要从头开始申请一个：[Google Cloud](https://cloud.google.com/?hl=zh-cn)。有账号之后，进入控制台，然后选择“API和服务”，点击“库”，进入库页面，搜索drive，启用该API，如果还没有创建过项目，那么需要先创建一个项目，这样才能够启用API。

选择“API和服务”，点击“凭据”进入页面，点击“创建凭据”，选择“OAuth客户端ID”，创建一个“OAuth 2.0 客户端 ID”，中间会让用户开启一个“OAuth同意屏幕”，会让选择“外部”还是“内部”用户，我的账号“内部”选择不了，所以就选择外部了，随便输入一个项目名称即可，然后在“Google API 的范围”里面需要把google drive添加进去：https://www.googleapis.com/auth/drive， 然后开启即可。

然后回到“创建凭据”，选择“web应用”，随便命名即可，然后在 "已获授权的 JavaScript 来源" URI里添加： http://localhost:8080  ，在“”添加：http://localhost:8080/ ，注意最后一个/不能少，否则会查很久都不知道错哪，点击创建，即可得到一个OAuth 2.0 客户端 ID。这时候就有一个OAuth 2.0 客户端 ID条目出现了，点击右边下载按键即可下载文件，然后重命名为 client_secrets.json。

将该文件放到目前的目录下，就可以使用pydrive代码了。第一次使用LocalWebserverAuth()函数时，浏览器会提示自己有风险，为了使用drive API 需要暂时忽略该风险，因此在弹出的页面里，选择“高级选项”，同意项目访问drive即可。

可以生成一个本地凭证，这样不用总是需要远程认证，方法参考了：[Automating pydrive verification process](https://stackoverflow.com/questions/24419188/automating-pydrive-verification-process/24542604#24542604)的高票回答，即通过下面的执行语句，利用gauth.SaveCredentialsFile("mycreds.txt")生成一个本地的credits文件：

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()
gauth.LocalWebserverAuth()
gauth.SaveCredentialsFile("mycreds.txt")
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=893841030002-hsmvq01daf8kk45diqjth149bq120pk3.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


接下来就可以很简单地实现文件的上传和下载了。首先上传一个文件：

In [3]:
file1 = drive.CreateFile({'title': 'Hello.txt'})
file1.SetContentString('Hello')
file1.Upload() # Files.insert()

然后可以直接修改该文件名称：

In [3]:
file1['title'] = 'HelloWorld.txt'  # Change title of the file
file1.Upload() # Files.patch()

接下来可以试试往文件中增加内容。

In [4]:
content = file1.GetContentString()  # 'Hello'
file1.SetContentString(content+' World!')  # 'Hello World!'
file1.Upload() # Files.update()

下载文件使用GetContentFile(filename)或者 GetContentString().

In [5]:
file2 = drive.CreateFile({'id': file1['id']})
print('Downloading file %s from Google Drive' % file2['title'])
type(file1['id'])

str

Save Drive file as a local file，参数是本地文件的名称

In [7]:
file2.GetContentFile('HelloWorld.txt')  

还可以查询指定目录下的文件

In [8]:
# Auto-iterate through all files that matches this query
file_list1 = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list1:
    print('title: %s, id: %s' % (file1['title'], file1['id']))
    if file1['title']=='test':
        file1_id=str(file1['id'])
        #  列出某个文件夹下的文件        
        print(file1_id)
        file_list = drive.ListFile({'q': "'"+file1_id+"' in parents and trashed=false"}).GetList()
        for file in file_list:
            print('title: %s, id: %s' % (file['title'], file['id']))

title: HelloWorld.txt, id: 1g4mT39PcEQEs17oVOIIKRRQFLH0G4Vuc
title: refactor the codes, id: 1sSuNTbyftvZmXFthxtweBPGBBQH8IEDyABiKg4M6bG8
title: WRData, id: 1Ra-uA7lT_HBvPI2afQkYxXxjkWOuRRuQ
title: test, id: 1jhJpA9qKSmY0bi3jOvMl9FaGLcKIfN2I
1jhJpA9qKSmY0bi3jOvMl9FaGLcKIfN2I
title: HelloWorld.txt, id: 1HJOkPe0dlrU_97T6h6QxvNmYGhbihVdP
title: foo.txt, id: 1RdQHHeHsHakuyCLzguC7Y36R7DxUIzF0
title: daymet, id: 1bFV4R6FW5WUlUoVkLsO25YhUzeeFvHCA
title: RS, id: 1G1a9rbPsHnbJ6xhlznKB0U_UrxwXrWw0
title: drive, id: 1-4LOn3r-drroBNMYwEA7HBMZNmgW5pM1
title: Colab Notebooks, id: 1dtSHtNnHBRAJo5xjBM2CEmwYETU3BovM
title: Life, id: 1Oml-Ia5ricBjFNT2yV-m1D9Nhoch0Ohy
title: Research, id: 1UTOQVqsqqzbOBGRW7KeLs91GDDS1JT7g


从上可知，下载一个文件需要指定其id。如何查看某个文件夹下的id呢？，需要在ListFIle函数中使用某个文件夹的id，这样可以如下列出文件夹下的文件。

In [9]:
# Auto-iterate through all files that matches this query
file_list = drive.ListFile({'q': "'1jhJpA9qKSmY0bi3jOvMl9FaGLcKIfN2I' in parents and trashed=false"}).GetList()
for file in file_list:
    print('title: %s, id: %s' % (file['title'], file['id']))

title: HelloWorld.txt, id: 1HJOkPe0dlrU_97T6h6QxvNmYGhbihVdP
title: foo.txt, id: 1RdQHHeHsHakuyCLzguC7Y36R7DxUIzF0


现在指定下载一个文件

In [10]:
file3 = drive.CreateFile({'id': '1HJOkPe0dlrU_97T6h6QxvNmYGhbihVdP'})
print('Downloading file %s from Google Drive' % file3['title'])
file3.GetContentFile(file3['title'])  

批量下载文件可使用下列代码：

In [12]:
# Auto-iterate through all files that matches this query
import os
file_list = drive.ListFile({'q': "'1jhJpA9qKSmY0bi3jOvMl9FaGLcKIfN2I' in parents and trashed=false"}).GetList()
for file in file_list:
    print('title: %s, id: %s' % (file['title'], file['id']))
    file_dl= drive.CreateFile({'id':file['id']})
    print('Downloading file %s from Google Drive' % file_dl['title'])
    # 下载到指定文件夹    
    directory="./test"
    file_dl.GetContentFile(os.path.join(directory,file_dl['title']))  
print('Downloading file finished')

title: HelloWorld.txt, id: 1HJOkPe0dlrU_97T6h6QxvNmYGhbihVdP
title: foo.txt, id: 1RdQHHeHsHakuyCLzguC7Y36R7DxUIzF0


以上就是简单利用PyDrive进行文件下载的方式。